This notebook demonstrates the task of image classification, in this case a classification of: reservoir | natural waterbody | shadows. The notebook includes the fine-tuning (training phase), testing and inference on a dataset of satellite images containing waterbodies in Angola. The fine-tuning is performed on a pre-trained CNN.

 The notebook follows these steps:

*   General setup of DL environment in Google Colab (coupled to your Google Drive)
*   All settings
*   Fine-tuning the model with a training dataset
*   Testing the fine-tuned model
*   Inference: classification of waterbodies




# General setup

In [ ]:
# !pip install --upgrade geopandas

# Imports
import pathlib
import numpy as np
import os
import sys
import datetime
import itertools
import tensorflow_hub as hub
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
import geopandas as gpd
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
# The following line of code is a shell command executed within the notebook. It uses the '!' prefix to run a command as if it were in the terminal.
!nvidia-smi

# The 'nvidia-smi' command stands for NVIDIA System Management Interface. It is used to query and manage NVIDIA GPU devices.
# Running this command displays information about the GPUs on your system, including utilization, memory usage, driver version, and more.

In [ ]:
# Mount Google Drive to access your (training, test and inference) data (you have to consent to everything to get it working..)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Set the AUTOTUNE option for automatic optimization and print the Tensorflow version
AUTO = tf.data.experimental.AUTOTUNE
print(f"Tensorflow version that is currently being used: {tf.__version__}")

In [ ]:
# Initialize TPU if data is stored in the cloud
tpu = None

if tpu:
  # If TPU is initialized, shut down the TPU system
  tf.tpu.experimental.shutdown_tpu_system(tpu)

try:
  # Attempt to detect and initialize a TPU
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
  # If no TPU is found, fall back to default strategy for CPU or single GPU
  strategy = tf.distribute.get_strategy() # default strategy for CPU and single GPU

# Settings

#### Paths and files

In [ ]:
# Google Drive directory that stores the downloaded data and scripts (later changeo to name repository folder: image_classification)
dir_gdrive = r'/content/drive/MyDrive/AngolaTrainingData'

# Datasets, model and scripts
dir_data = f'{dir_gdrive}/00_data'
dir_model = f'{dir_gdrive}/01_model'
dir_scripts = f'{dir_gdrive}/02_src'
sys.path.append(dir_scripts)

# Coordinates of waterbodies
filename_geojson = 'blobdetected_angola_blobs_filtered_full'

# Print system path
sys.path

#### Model (pre-trained CNN)

In [ ]:
model_name = "efficientnetv2-xl-21k"
model_handle = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_xl/feature_vector/2"

#### Training

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1.5 / 255)
do_data_augmentation = True

IMAGE_SIZE = (512, 512) # this is the image shape required for the keras model (or what): so if the images have different shapes (that is the case for the vortex images. TODO: the images need to be reshaped to (512, 512); there is a keras function for this)
BATCH_SIZE = 16         # TODO: check with Antonio: he used batch size 20 for the testing and inference
EPOCHS = 300

drop_out_rate = 0.3
learning_rate = 0.005
momentum = 0.9
label_smoothing = 0.1
metrics = ['accuracy']

# Fine-tune model

## Training

### Build dataset

In [ ]:
from dataset import build_training_dataset, augment_data

# Training data
dir_data_training = os.path.join(dir_data, 'training') # dir_data_training = f'{dir_gdrive}/AngolaFullTraining'
ds_train_r, class_names, ds_train_size = build_training_dataset(dir_data_training, "training", IMAGE_SIZE, BATCH_SIZE)
preprocessing_model = tf.keras.Sequential([normalization_layer])
preprocessing_model = augment_data(do_data_augmentation, preprocessing_model)
ds_train = ds_train_r.map(lambda images, labels:(preprocessing_model(images), labels))

# Validation data
ds_val, class_names, ds_val_size = build_training_dataset(dir_data_training, "validation", IMAGE_SIZE, BATCH_SIZE)
ds_val = ds_val.map(lambda images, labels:(normalization_layer(images), labels))

In [ ]:
from dataset import plot_training_data

# Plot a few images from the training dataset to check
fig_train = plot_training_data(ds_train, 8)

In [ ]:
# Plot a few images from the validation dataset to check
fig_val = plot_training_data(ds_val, 8)

### Build model

In [ ]:
# Composition of the model of different layers (input layer, original encoder layers, dense layers that are trained with the satellite imagery dataset)
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=IMAGE_SIZE + (3,)),
    hub.KerasLayer(model_handle, trainable=False),
    tf.keras.layers.Dropout(rate=drop_out_rate),
    tf.keras.layers.Dense(len(class_names),
    kernel_regularizer=tf.keras.regularizers.l2(0.0001))
])
model.build((None,)+IMAGE_SIZE+(3,))
model.summary()

# Configure the model for training by specifiying the optimizer (Stochastic Gradient Descent), loss function and metrics to be used
model.compile(
  optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum),
  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=label_smoothing),
  metrics=metrics)

### Train model

In [ ]:
# Actual training of the dense layers in the sequenced model
hist = model.fit(
    ds_train,
    epochs=EPOCHS, steps_per_epoch=ds_train_size // BATCH_SIZE,
    validation_data=ds_val,
    validation_steps=ds_val_size // BATCH_SIZE).history

In [ ]:
# Plot results of the training
from fine_tune import plot_training_results

fig = plot_training_results(hist)

In [ ]:
# Save the fine-tuned model (and its history object) to use for inference
from fine_tune import save_model

save_model(model, dir_model, model_name, hist)

## Testing

### Build dataset

In [ ]:
dir_data_testing = os.path.join(dir_data, 'testing') # dir_data_testing = f'{dir_gdrive}/testing3class'

ds_test = tf.keras.utils.image_dataset_from_directory(
    dir_data_testing,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE, #25?
    shuffle = True
)

ds_test = ds_test.map(lambda images, labels:(normalization_layer(images), labels))

### Test model

In [ ]:
# Load fine-tuned model
# model = keras.models.load_model(os.path.join(dir_model, f'{model_name}_finetuned.h5'),
#        custom_objects={'KerasLayer':hub.KerasLayer})
model = keras.models.load_model(r'/content/drive/MyDrive/AngolaTrainingData/model_EffNetV2_retrainedAngola_3class.h5',
       custom_objects={'KerasLayer':hub.KerasLayer})

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
from testing import test_model

labels_test, predictions_test = test_model(model, ds_test)

acc = accuracy_score(labels_test, predictions_test)
cm = confusion_matrix(labels_test, predictions_test)

print(f"Accuracy: {acc}")
print(f"Confusion matrix: \n{cm}")

In [ ]:
from plot import plot_test_results

fig = plot_test_results(model, ds_test)

# Inference

### Build dataset

In [ ]:
from inference import get_img_ids

dir_data_inference = os.path.join(dir_data, 'inference') # dir_data_inference = f'{dir_gdrive}/AngolaPNGsZoom'

ds_inf = tf.keras.utils.image_dataset_from_directory(
    dir_data_inference,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE, #20?
    shuffle = False,
    label_mode=None,
    labels=None)

img_ids = get_img_ids(ds_inf)

ds_inf = ds_inf.map(lambda images:(normalization_layer(images)))

### Inference

In [ ]:
predictions = model.predict(dataset)
y_pred = np.argmax(predictions, axis = 1)

In [ ]:
# Show some results of the inference
from inference import plot_inference_results

fig = plot_inference_results(model, ds_inf)

In [ ]:
# Link the waterbodies in the images to the coordinates of the blobs in the geojson file and save to a new geojson file
from inference import link_waterbodies_coordinates, save_prediction_geojson

data_coord = gpd.read_file(os.path.join(dir_data, f'{filename_geojson}.geojson'))
pred_coord = linke_waterbodies_coordinates(img_ids, y_pred, data_coord)
save_prediction_geojson(pred_coord, dir_data, filename_geojson)